# 02 - Preparación de datos y creación de ventanas para LSTM


In [3]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler

In [4]:
ruta = "../data/raw/continuous_dataset.csv"

df = pd.read_csv(
    ruta,
    parse_dates=['datetime'],
    index_col='datetime'
).sort_index()


In [5]:
df['hour'] = df.index.hour
df['dayofweek'] = df.index.day_of_week
df['is_weekend'] = (df['dayofweek'] >= 5).astype(int)

feature_cols = [
    'nat_demand',
    'T2M_toc', 'T2M_san', 'T2M_dav',
    'hour', 'dayofweek', 'is_weekend',
    'holiday', 'school'
]

target_col = 'nat_demand'

df_model = df[feature_cols].copy()
df_model.head()

,nat_demand,T2M_toc,T2M_san,T2M_dav,hour,dayofweek,is_weekend,holiday,school
datetime,,,,,,,,,
2015-01-03 01:00:00,970.3450,25.865259,23.482446,22.662134,1,5,1,0,0
2015-01-03 02:00:00,912.1755,25.899255,23.399255,22.578943,2,5,1,0,0
2015-01-03 03:00:00,900.2688,25.937280,23.343530,22.531030,3,5,1,0,0
2015-01-03 04:00:00,889.9538,25.957544,23.238794,22.512231,4,5,1,0,0
2015-01-03 05:00:00,893.6865,25.973840,23.075403,22.481653,5,5,1,0,0


In [6]:
n = len(df_model)
train_size = int(n * 0.8)

df_train = df_model.iloc[:train_size]
df_test  = df_model.iloc[train_size:]

print("Train:", df_train.index.min(), "→", df_train.index.max(), "| filas:", len(df_train))
print("Test: ", df_test.index.min(), "→", df_test.index.max(),  "| filas:", len(df_test))

Train: 2015-01-03 01:00:00 → 2019-05-23 14:00:00 | filas: 38438
Test:  2019-05-23 15:00:00 → 2020-06-27 00:00:00 | filas: 9610


In [7]:
X_train = df_train[feature_cols].values
y_train = df_train[[target_col]].values   # doble [] -> 2D

X_test  = df_test[feature_cols].values
y_test  = df_test[[target_col]].values

scaler_X = MinMaxScaler()
scaler_y = MinMaxScaler()

X_train_scaled = scaler_X.fit_transform(X_train)   # fit SOLO en train
y_train_scaled = scaler_y.fit_transform(y_train)

X_test_scaled  = scaler_X.transform(X_test)        # usamos los mismos params
y_test_scaled  = scaler_y.transform(y_test)

print(X_train_scaled.shape, y_train_scaled.shape)

(38438, 9) (38438, 1)


In [8]:
def create_sequences(X, y, seq_len):
    Xs, ys = [], []
    for i in range(seq_len, len(X)):
        Xs.append(X[i-seq_len:i])  # [t-seq_len, ..., t-1]
        ys.append(y[i])            # valor en t
    return np.array(Xs), np.array(ys)


In [9]:
sequence_length = 24 * 7  # 7 días de historia (168 horas)

X_train_seq, y_train_seq = create_sequences(X_train_scaled, y_train_scaled, sequence_length)
X_test_seq,  y_test_seq  = create_sequences(X_test_scaled,  y_test_scaled,  sequence_length)

print("X_train_seq:", X_train_seq.shape)
print("y_train_seq:", y_train_seq.shape)
print("X_test_seq :", X_test_seq.shape)
print("y_test_seq :", y_test_seq.shape)

X_train_seq: (38270, 168, 9)
y_train_seq: (38270, 1)
X_test_seq : (9442, 168, 9)
y_test_seq : (9442, 1)


In [10]:
import torch
from torch.utils.data import TensorDataset, DataLoader

device = "cuda" if torch.cuda.is_available() else "cpu"
print("Usando device:", device)

X_train_tensor = torch.tensor(X_train_seq, dtype=torch.float32).to(device)
y_train_tensor = torch.tensor(y_train_seq, dtype=torch.float32).to(device)

X_test_tensor  = torch.tensor(X_test_seq,  dtype=torch.float32).to(device)
y_test_tensor  = torch.tensor(y_test_seq,  dtype=torch.float32).to(device)

X_train_tensor.shape, y_train_tensor.shape


Usando device: cuda


(torch.Size([38270, 168, 9]), torch.Size([38270, 1]))

In [17]:
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)

batch_size = 64
train_loader = DataLoader(train_dataset, batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size, shuffle=False)


In [12]:
import torch.nn as nn 

class LSTMForecast(nn.Module): 
    def __init__(self, input_size, hidden_size=64, num_layers=2, output_size=1):
        super().__init__()
        # Capa de LSTM
        self.lstm = nn.LSTM(
            input_size=input_size,
            hidden_size=hidden_size,
            num_layers=num_layers,
            batch_first=True
        )
        # Capa lineal final 
        self.fc = nn.Linear(hidden_size, 1)
    
    def forward(self, x):
        # x: (batch_size, seq_len, input_size)
        out, _ = self.lstm(x)

        last_hidden = out[:, -1, :]
        out = self.fc(last_hidden)
        return out
        

Tengo un batch de 64 ventanas.
Cada ventana representa 7 días (168 horas) y, en cada hora, tengo 9 features.

El LSTM es un solo modelo (no 64 distintos) que procesa las 64 ventanas en paralelo.
Si hago zoom en una sola ventana:

el LSTM recorre las 168 horas en orden temporal,

en cada paso ve un vector de 9 features (esa hora) y actualiza un estado interno (hidden) de tamaño 64,

al terminar la hora 168, me quedo con el último hidden, que es un resumen de lo que pasó en esas 168 horas.

Ese vector de tamaño 64 lo paso por una capa final (fc) para obtener una predicción de la demanda en la hora siguiente.

Esto mismo pasa para las 64 ventanas del batch en paralelo, así que obtengo 64 predicciones.
Comparo esas 64 predicciones con los 64 valores reales (y_batch), calculo la pérdida (por ejemplo el MSE promedio) y con eso ajusto los pesos del LSTM y de la capa final para que la próxima vez prediga mejor.

In [21]:
input_size = X_train_seq.shape[2]

model = LSTMForecast(input_size).to(device)

model

criterion = nn.MSELoss()

optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

print(model)


LSTMForecast(
  (lstm): LSTM(9, 64, num_layers=2, batch_first=True)
  (fc): Linear(in_features=64, out_features=1, bias=True)
)


In [23]:
n_epochs = 15

for epoch in range(n_epochs):
    #train
    model.train()
    train_loss = 0.0

    for X_batch, y_batch in train_loader:
        optimizer.zero_grad() # limpiar gradientes anteriores
        y_pred = model(X_batch) # forward: LSTM + fc
        loss = criterion(y_pred, y_batch) # MSE entre preds y targets
        loss.backward() # backprop: calcula gradientes
        optimizer.step() # actualiza pesos del modelo
        
        train_loss += loss.item() * X_batch.size(0)
    train_loss /= len(train_loader.dataset)

    # Validation 
    model.eval()
    val_loss = 0.0
    with torch.no_grad():
        for X_batch, y_batch in test_loader:
            y_pred = model(X_batch)
            loss = criterion(y_pred, y_batch)
            val_loss += loss.item() * X_batch.size(0)

    val_loss /= len(test_loader.dataset)

    print(f"Epoch {epoch+1}: train_loss={train_loss:.6f}, val_loss={val_loss:.6f}")

Epoch 1: train_loss=0.000195, val_loss=0.000337
Epoch 2: train_loss=0.000189, val_loss=0.000282
Epoch 3: train_loss=0.000187, val_loss=0.000289
Epoch 4: train_loss=0.000183, val_loss=0.000281
Epoch 5: train_loss=0.000181, val_loss=0.000278
Epoch 6: train_loss=0.000183, val_loss=0.000283
Epoch 7: train_loss=0.000178, val_loss=0.000322
Epoch 8: train_loss=0.000173, val_loss=0.000359
Epoch 9: train_loss=0.000180, val_loss=0.000306
Epoch 10: train_loss=0.000176, val_loss=0.000311
Epoch 11: train_loss=0.000178, val_loss=0.000293
Epoch 12: train_loss=0.000173, val_loss=0.000273
Epoch 13: train_loss=0.000172, val_loss=0.000260
Epoch 14: train_loss=0.000173, val_loss=0.000294
Epoch 15: train_loss=0.000168, val_loss=0.000310
